In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

Connecting to SQL database

In [2]:
import sqlite3

conn = sqlite3.connect("news_data.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS news_articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company TEXT,
    source TEXT,
    title TEXT,
    description TEXT,
    url TEXT UNIQUE,
    published_at TEXT
)
""")

conn.commit()
conn.close()

Importing news API

In [4]:
from newsapi.newsapi_client import NewsApiClient
import os
from datetime import datetime, timedelta

api = NewsApiClient(api_key=os.getenv("NEWS_API_KEY"))


company = 'Reliance'
query = 'Reliance Industries'



to_date = datetime.utcnow()
from_date = to_date - timedelta(days=30)

response = api.get_everything(
    q=query,
    language="en",
    sort_by="publishedAt",
    from_param=from_date.strftime("%Y-%m-%d"),
    to=to_date.strftime("%Y-%m-%d"),
    page_size=100
)

In [5]:
def store_news_sqlite(company, articles):
    conn = sqlite3.connect("news_data.db")
    cursor = conn.cursor()

    for article in articles:
        try:
            cursor.execute("""
            INSERT OR IGNORE INTO news_articles
            (company, source, title, description, url, published_at)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (
                company,   
                article.get("source", {}).get("name"),
                article.get("title"),
                article.get("description"),
                article.get("url"),
                article.get("publishedAt")
            ))
        except Exception as e:
            print("Error inserting article:", e)

    conn.commit()
    conn.close()

In [6]:
response = api.get_everything(q=query, language="en", sort_by="publishedAt")
store_news_sqlite(company, response["articles"])


In [8]:
import pandas as pd
import sqlite3

company = ['Reliance']
conn = sqlite3.connect("news_data.db")
placeholders = ",".join(["?"] * len(company))
query = f"""
SELECT *
FROM news_articles
WHERE company in ({placeholders})
"""

df = pd.read_sql(query, conn, params=company)

conn.close()

df.head()

,id,company,source,title,description,url,published_at
0,4844,Reliance,The Times of India,View: India is cornered by China’s lock on tech,India-China trade: India's industrial growth f...,https://economictimes.indiatimes.com/industry/...,2026-01-21T01:14:07Z
1,4845,Reliance,OilPrice.com,India Deepens Energy Ties With the Gulf While ...,India’s energy strategy is increasingly define...,https://oilprice.com/Energy/Energy-General/Ind...,2026-01-21T01:00:00Z
2,4846,Reliance,BusinessLine,Mixed economic outlook,"Growth is strong, but global headwinds, a concern",https://www.thehindubusinessline.com/opinion/m...,2026-01-21T00:30:00Z
3,4847,Reliance,Livemint,Early Q3 earnings show costs overtaking revenu...,A jump in labour and input costs has flipped I...,https://www.livemint.com/companies/early-q3-ea...,2026-01-21T00:20:42Z
4,4848,Reliance,GlobeNewswire,Copperhead Announces Approval of Change of Bus...,"Vancouver, BC, Jan. 20, 2026 (GLOBE NEWSWIRE) ...",https://www.globenewswire.com/news-release/202...,2026-01-20T23:58:00Z
